# `H2MM_C` Displaying Optimization Progress

Let's get our obligitory imports in order, and we'll load the 3 detector data as well.

In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt

import H2MM_C as hm

# load the data
color3 = list()
times3 = list()

i = 0
with open('sample_data_3det.txt','r') as f:
    for line in f:
        if i % 2 == 0:
            times3.append(np.array([int(x) for x in line.split()],dtype='Q'))
        else:
            color3.append(np.array([int(x) for x in line.split()],dtype='L'))
        i += 1

## Customizing Display of Optmization progression

> Note
>
> These functions are best demonstrated in Jupyter notebooks

You've probably seen that whenever `hm.EM_H2MM_C()` is run, you see the number of iterations currently computed gets displayed. This behavior can be modified however with the `print_func` keyword argument. This lets us choose how much/what information gets displayed! See the list of options below:


### Basic `print_func` options

- `'iter'` Prints only the iteration number- compact way to still track optimization progress in Jupyter Notebook
- `'all'` prints a representation of the whole model to be optimized in the next iteration (this will be very verbose).
- `'diff'` Print the difference between the previous and current model logliklihoods and the current loglikelihood.
- `'diff_time'` Same as `'diff'` but with additional information about how long the current iteration took, and the total time taken. These times however are not very accurate, because they use an inaccurate, but fast clock function.
- `'comp'` Print the old and current loglikelihoods
- `'comp_time'` Like `'diff_time'` but for `'comp'`
- `'console'` (not recomended anymore) normally prints to the **console** window, and provides basic information on the progress of the optimization.
    > This option may not work on your computer due to how the compilation works and different systems having different default console outputs
- `None` Suppresses all printing of iteration information.

In [2]:
model_3d3c = hm.EM_H2MM_C(hm.factory_h2mm_model(3,3), color3,  times3, print_func='diff')

The model converged after 191 iterations

These updates can be further customized. By default only the current itteration is displayed, but what if you would like to record the progress, for instance to see how the loglik improves with each iteration?

This can be done by passing another keyword argument: `print_args=True`

#### Recording each iteration with the `print_args` kwarg:

In [3]:
model_3d3c = hm.EM_H2MM_C(hm.factory_h2mm_model(3,3), color3,  times3, print_func='diff', print_args=True)

Iteration:    0, loglik:-4.387899e+05, improvement:   inf
Iteration:    1, loglik:-4.203991e+05, improvement:1.839076e+04
Iteration:    2, loglik:-4.172495e+05, improvement:3.149621e+03
Iteration:    3, loglik:-4.168160e+05, improvement:4.334909e+02
Iteration:    4, loglik:-4.166697e+05, improvement:1.463433e+02
Iteration:    5, loglik:-4.165714e+05, improvement:9.831467e+01
Iteration:    6, loglik:-4.164868e+05, improvement:8.460410e+01
Iteration:    7, loglik:-4.164030e+05, improvement:8.370726e+01
Iteration:    8, loglik:-4.163116e+05, improvement:9.142352e+01
Iteration:    9, loglik:-4.162055e+05, improvement:1.061152e+02
Iteration:   10, loglik:-4.160785e+05, improvement:1.269588e+02
Iteration:   11, loglik:-4.159256e+05, improvement:1.529380e+02
Iteration:   12, loglik:-4.157447e+05, improvement:1.808611e+02
Iteration:   13, loglik:-4.155401e+05, improvement:2.046643e+02
Iteration:   14, loglik:-4.153189e+05, improvement:2.211944e+02
Iteration:   15, loglik:-4.150866e+05, improve

`print_args` can also be used to specify how frequently the display updates, by passing an *integer* value into `print_args`, then the display will only update after that many iterations, which means it stays around for longer, 
so you can actually read the full line before it updates.

Update the display every 10 iterations:

In [4]:
model_3d3c = hm.EM_H2MM_C(hm.factory_h2mm_model(3,3), color3,  times3, print_func='diff', print_args=10)

The model converged after 191 iterations

These options can be combined, using a tuple of precicely 2 elements, the first must be a positive `int` and the second must be either `True` or `False`. So `print_args=(10,True)` will update every 10 iterations, and keep the record of each iteration, while `print_args=(20,False)` will update the display every 20 iterations, and only display the latest iteration.

Show every 10 minuts and keep previous printout:

In [5]:
model_3d3c = hm.EM_H2MM_C(hm.factory_h2mm_model(3,3), color3,  times3, print_func='diff', print_args=(10, True))

Iteration:    0, loglik:-4.387899e+05, improvement:   inf
Iteration:   10, loglik:-4.160785e+05, improvement:1.269588e+02
Iteration:   20, loglik:-4.138319e+05, improvement:2.561196e+02
Iteration:   30, loglik:-4.114426e+05, improvement:2.344301e+02
Iteration:   40, loglik:-4.097127e+05, improvement:9.049932e+01
Iteration:   50, loglik:-4.094081e+05, improvement:7.740962e+00
Iteration:   60, loglik:-4.093820e+05, improvement:7.351031e-01
Iteration:   70, loglik:-4.093794e+05, improvement:7.134807e-02
Iteration:   80, loglik:-4.093792e+05, improvement:7.217587e-03
Iteration:   90, loglik:-4.093792e+05, improvement:8.952386e-04
Iteration:  100, loglik:-4.093791e+05, improvement:1.619327e-04
Iteration:  110, loglik:-4.093791e+05, improvement:4.142115e-05
Iteration:  120, loglik:-4.093791e+05, improvement:1.258310e-05
Iteration:  130, loglik:-4.093791e+05, improvement:4.038040e-06
Iteration:  140, loglik:-4.093791e+05, improvement:1.323235e-06
Iteration:  150, loglik:-4.093791e+05, improve

### Customized display using your own function

If you want to customize the display, you can define  your own printer function.

The function should have the following general signature:

`print_func(niter, new_model, current_model, old_model, t_iter, t_total)`

> NOTE: It is not necessary, but recommended to keep these variable names in the function declaration.

where:
- `niter` is the number of iterations
- `new_model` is a `h2mm_model` object that represents the next model to be optimized (**before** checking for out of bounds values) note that its `.loglik` will be irrelevant because it has not been calculated yet.
- `current_model` is a `h2mm_model` object that represents the model whose `.loglik` was just calculated
- `old_model` is a `h2mm_model` object that represents the model from the previous iteration.
- `t_iter` is a float which is the time in seconds based on the **inaccurate C clock** that it took to calculate the latest iteration
- `t_total` is a float which is the time in seconds based on the **inaccurate C clock** that the full optimization has taken

Your function can return a string, which is the prefered method, or you can call `print` from within the function.

The `print_args` keyword argument works in the same way as before.

So bellow is an example of a custom print function:

In [6]:
def silly_print(niter, new, current, old, titer, time):
    return f"""We haven't finished after {niter} iterations
    with {new.loglik - current.loglik} improvement in loglik 
    after {time} (inaccurate) seconds"""

In [7]:
hm.EM_H2MM_C(hm.factory_h2mm_model(3,3), color3, times3, print_func=silly_print, print_args=(10, False))

The model converged after 192 iterations

nstate: 3, ndet: 3, nphot: 436084, niter: 192, loglik: -409379.14701996546 converged state: 3
prior:
0.2599161389040611, 0.49437669708180326, 0.24570716401413564
trans:
0.9999762009998817, 2.109153713283627e-05, 2.707462985396962e-06
8.392833666553347e-06, 0.9999812080745007, 1.0399091832742116e-05
6.290339969797629e-06, 4.466207443278959e-05, 0.9999490475855974
obs:
0.14570425052915176, 0.29344316376623253, 0.5608525857046157
0.44173952091701757, 0.08763105664801644, 0.4706294224349659
0.8414287396718038, 0.0785286841458778, 0.08004257618231851

If your function calls `print` directly (and potentially you could use other display features). This will be more direct, but `print_args` arguments will be ignored.

In [8]:
def silly_fix_print(niter, new, current, old, titer, time, *args):
    print( f"""We haven't finished after {niter} iterations
    with {new.loglik - current.loglik} improvement in loglik 
    after {time} (inaccurate) seconds, {args}""")

In [9]:
hm.EM_H2MM_C(hm.factory_h2mm_model(3,3), color3, times3, 
             print_func=silly_fix_print, print_args=(75, True, "eggs"))

The model converged after 191 iterations

We haven't finished after 1 iterations
    with 0.0002 improvement in loglik 
    after 0.2 (inaccurate) seconds, ('eggs',)
We haven't finished after 0 iterations
    with 29410.734241753118 improvement in loglik 
    after 0.116358 (inaccurate) seconds, ('eggs',)
We haven't finished after 75 iterations
    with 409379.2380262765 improvement in loglik 
    after 8.661903 (inaccurate) seconds, ('eggs',)
We haven't finished after 150 iterations
    with 409379.1470236728 improvement in loglik 
    after 17.169087 (inaccurate) seconds, ('eggs',)


nstate: 3, ndet: 3, nphot: 436084, niter: 191, loglik: -409379.14701996325 converged state: 3
prior:
0.25991614498503507, 0.4943766688551135, 0.2457071861598515
trans:
0.99997620100061, 2.1091530647357813e-05, 2.7074687426012225e-06
8.392831156401862e-06, 0.9999812080804615, 1.0399088382200984e-05
6.290351439278532e-06, 4.466205603341905e-05, 0.9999490475925273
obs:
0.14570425274135482, 0.2934431598919638, 0.5608525873666814
0.441739522150704, 0.08763105601610267, 0.47062942183319323
0.841428729938364, 0.07852868414074789, 0.0800425859208882

### Extra args in `print_func`

If `print_args` gets a tuple with more than 2 arguments, it will pass the remaining arguments as \*args:

In [10]:
def silly_print(niter, new, current, old, titer, time, args):
    return f"""
    We haven't finished after {niter} iterations
    with {new.loglik - current.loglik} improvement in loglik 
    after {time} (inaccurate) seconds, {args}"""

In [11]:
hm.EM_H2MM_C(hm.factory_h2mm_model(3,3), color3, times3, 
             print_func=silly_print, print_args=(75, True, "I'm very silly"))


    We haven't finished after 0 iterations
    with 438789.88126171695 improvement in loglik 
    after 0.120712 (inaccurate) seconds, I'm very silly
    We haven't finished after 75 iterations
    with 409379.238026278 improvement in loglik 
    after 8.561962 (inaccurate) seconds, I'm very silly
    We haven't finished after 150 iterations
    with 409379.1470236733 improvement in loglik 
    after 17.064443 (inaccurate) seconds, I'm very sillyThe model converged after 192 iterations

nstate: 3, ndet: 3, nphot: 436084, niter: 192, loglik: -409379.1470199657 converged state: 3
prior:
0.2599161389040611, 0.4943766970818051, 0.2457071640141338
trans:
0.9999762009998817, 2.1091537132836017e-05, 2.7074629853969673e-06
8.392833666553302e-06, 0.9999812080745007, 1.0399091832742082e-05
6.290339969797655e-06, 4.4662074432789244e-05, 0.9999490475855974
obs:
0.14570425052915192, 0.2934431637662306, 0.5608525857046175
0.44173952091701685, 0.08763105664801694, 0.4706294224349663
0.8414287396718046, 0.07852868414587744, 0.0800425761823178